# train

## load library

In [1]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter

from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import mlflow
from collections import defaultdict
from scipy.stats import zscore

# original library

import common as com
import pytorch_modeler as modeler
from pytorch_model import CNN6PANNsAutoEncoder as Model
from pytorch_utils import filtered_load_model
import models

import librosa
import IPython
import librosa.display

## load config and set logger

In [2]:
with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [3]:
# Setting seed
modeler.set_seed(42)

In [4]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_path = INPUT_ROOT + "/dev_data"
add_dev_path = INPUT_ROOT + "/add_dev_data"
# machine type
MACHINE_TYPE = config['IO_OPTION']['MACHINE_TYPE']
machine_types = os.listdir(dev_path)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
PKL_DIR = OUTPUT_ROOT + '/pkl'
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(PKL_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/working/research/dcase2020/result/2D/CONV_AE_transfer/config.yaml'

## make path list and train/valid split

In [5]:
############################################################################
# make path set and train/valid split
############################################################################
'''
train_paths[machine_type]['train' or 'valid'] = path
'''
dev_train_paths = {}
add_train_paths = {}
train_paths = {}

In [6]:
for machine_type in machine_types:
    # dev train
    dev_train_all_paths = ["{}/{}/train/".format(dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(dev_path, machine_type))]
    dev_train_all_paths = sorted(dev_train_all_paths)
    dev_train_paths[machine_type] = {}
    dev_train_paths[machine_type]['train'], \
    dev_train_paths[machine_type]['valid'] = train_test_split(dev_train_all_paths,
                                                              test_size=config['etc']['test_size'],
                                                              shuffle=False,
                                                             )
    # add_dev train
    add_train_all_paths = ["{}/{}/train/".format(add_dev_path, machine_type) + file for file in os.listdir("{}/{}/train".format(add_dev_path, machine_type))]
    add_train_all_paths = sorted(add_train_all_paths)
    add_train_paths[machine_type] = {}
    add_train_paths[machine_type]['train'], \
    add_train_paths[machine_type]['valid'] = train_test_split(add_train_all_paths,
                                                              test_size=config['etc']['test_size'],
                                                              shuffle=False,
                                                             )
    train_paths[machine_type] = {}
    train_paths[machine_type]['train'] = dev_train_paths[machine_type]['train'] + add_train_paths[machine_type]['train']
    train_paths[machine_type]['valid'] = dev_train_paths[machine_type]['valid'] + add_train_paths[machine_type]['valid']

## training

In [7]:
#############################################################################
# run
#############################################################################
def run(machine_type):
    com.tic()
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(train_paths, machine_type)
    # define writer for tensorbord
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)         # debug
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    logger.info('TRAINING')
    # parameter setting
    net = Model(sample_rate=config['preprocessing']['sample_rate'],
            window_size=config['preprocessing']['window_size'],
            hop_size=config['preprocessing']['hop_size'],
            mel_bins=config['preprocessing']['mel_bins'],
            fmin=config['preprocessing']['fmin'],
            fmax=config['preprocessing']['fmax'])
    pretrained_dict = torch.load(config['IO_OPTION']['PREMODEL_PATH'])
    net = filtered_load_model(net, pretrained_dict)
    ### Encoder fraze ###
    for idx, param in enumerate(net.parameters()):
        if idx == 17:
            break
        param.requires_grad = False
    #####################
    optimizer = optim.Adam(net.parameters())
    criterion = nn.MSELoss()
    num_epochs = config['fit']['num_epochs']
    history = modeler.train_net(net, dataloaders_dict, criterion, optimizer, num_epochs, writer)
    # output
    model = history['model']
    model_out_path = MODEL_DIR+'/{}_model.pth'.format(machine_type)
    torch.save(model.state_dict(), model_out_path)
    logger.info('\n success:{0} \n'.format(machine_type) + \
                    'model_out_path ==> \n {0}'.format(model_out_path))
    #  close writer for tensorbord
    writer.close()
    #modeler.mlflow_log(history, config, machine_type, model_out_path, tb_log_dir)
    com.toc()
    return history

In [8]:
machine_types

['fan', 'pump', 'slider', 'ToyCar', 'ToyConveyor', 'valve']

In [9]:
for machine_type in machine_types:
    history = run(machine_type)
    with open('{}/{}_history.pkl'.format(PKL_DIR, machine_type), 'wb') as file:
        pickle.dump(history , file)

2020-10-16 09:39:52,328 - 00_train.py - INFO - TARGET MACHINE_TYPE: fan
2020-10-16 09:39:52,329 - 00_train.py - INFO - MAKE DATA_LOADER
2020-10-16 09:39:53,262 - 00_train.py - INFO - TRAINING
  0%|          | 0/143 [00:00<?, ?it/s]

use: cuda:0


100%|██████████| 62/62 [00:21<00:00,  2.95it/s]
2020-10-16 09:41:07,827 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:809.091459, valid_loss:630.983270
100%|██████████| 62/62 [00:21<00:00,  2.94it/s]
2020-10-16 09:42:21,169 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:403.863812, valid_loss:221.773001
100%|██████████| 62/62 [00:21<00:00,  2.94it/s]
2020-10-16 09:43:34,648 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:113.304860, valid_loss:51.324058
100%|██████████| 62/62 [00:21<00:00,  2.94it/s]
2020-10-16 09:44:48,246 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:33.445342, valid_loss:31.658215
100%|██████████| 62/62 [00:21<00:00,  2.83it/s]
2020-10-16 09:46:03,833 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:26.389248, valid_loss:31.973994
100%|██████████| 62/62 [00:21<00:00,  2.87it/s]
2020-10-16 09:47:19,452 - pytorch_modeler.py - INFO - Epoch 6/100:train_loss:25.498271, valid_loss:31.157889
100%|██████████| 62/62 [00:21<00:00,  2.90it/s]
2020-

100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:29:02,817 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:23.053641, valid_loss:28.273840
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:30:15,909 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:23.146492, valid_loss:28.456556
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:31:28,973 - pytorch_modeler.py - INFO - Epoch 42/100:train_loss:23.057782, valid_loss:29.085191
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:32:42,044 - pytorch_modeler.py - INFO - Epoch 43/100:train_loss:23.166327, valid_loss:28.033437
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:33:55,117 - pytorch_modeler.py - INFO - Epoch 44/100:train_loss:23.011703, valid_loss:28.175141
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 10:35:08,211 - pytorch_modeler.py - INFO - Epoch 45/100:train_loss:23.044470, valid_loss:28.152486
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020

100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 11:18:13,280 - pytorch_modeler.py - INFO - Epoch 80/100:train_loss:22.692056, valid_loss:28.170324
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 11:19:26,234 - pytorch_modeler.py - INFO - Epoch 81/100:train_loss:22.586075, valid_loss:27.600227
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020-10-16 11:20:39,196 - pytorch_modeler.py - INFO - Epoch 82/100:train_loss:22.371128, valid_loss:28.020892
100%|██████████| 62/62 [00:22<00:00,  2.74it/s]
2020-10-16 11:21:54,132 - pytorch_modeler.py - INFO - Epoch 83/100:train_loss:22.545004, valid_loss:28.127361
100%|██████████| 62/62 [00:21<00:00,  2.91it/s]
2020-10-16 11:23:10,814 - pytorch_modeler.py - INFO - Epoch 84/100:train_loss:22.358055, valid_loss:27.920021
100%|██████████| 62/62 [00:21<00:00,  2.90it/s]
2020-10-16 11:24:26,317 - pytorch_modeler.py - INFO - Epoch 85/100:train_loss:22.407978, valid_loss:27.370035
100%|██████████| 62/62 [00:20<00:00,  2.96it/s]
2020

...

use: cuda:0


100%|██████████| 55/55 [00:25<00:00,  2.14it/s]
2020-10-16 19:22:10,758 - pytorch_modeler.py - INFO - Epoch 1/100:train_loss:912.501362, valid_loss:728.398772
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 19:23:15,837 - pytorch_modeler.py - INFO - Epoch 2/100:train_loss:526.657884, valid_loss:318.886775
100%|██████████| 55/55 [00:18<00:00,  2.93it/s]
2020-10-16 19:24:21,064 - pytorch_modeler.py - INFO - Epoch 3/100:train_loss:195.081702, valid_loss:94.156228
100%|██████████| 55/55 [00:18<00:00,  2.93it/s]
2020-10-16 19:25:26,290 - pytorch_modeler.py - INFO - Epoch 4/100:train_loss:58.217749, valid_loss:34.325849
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 19:26:31,531 - pytorch_modeler.py - INFO - Epoch 5/100:train_loss:33.579580, valid_loss:27.567477
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 19:27:36,714 - pytorch_modeler.py - INFO - Epoch 6/100:train_loss:31.265124, valid_loss:27.211091
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-

100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:04:33,700 - pytorch_modeler.py - INFO - Epoch 40/100:train_loss:27.840579, valid_loss:24.234979
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:05:38,892 - pytorch_modeler.py - INFO - Epoch 41/100:train_loss:27.822851, valid_loss:24.719409
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:06:44,114 - pytorch_modeler.py - INFO - Epoch 42/100:train_loss:27.810363, valid_loss:24.412190
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:07:49,309 - pytorch_modeler.py - INFO - Epoch 43/100:train_loss:27.801377, valid_loss:24.371542
100%|██████████| 55/55 [00:18<00:00,  2.93it/s]
2020-10-16 20:08:54,555 - pytorch_modeler.py - INFO - Epoch 44/100:train_loss:27.663020, valid_loss:24.915250
100%|██████████| 55/55 [00:18<00:00,  2.93it/s]
2020-10-16 20:09:59,824 - pytorch_modeler.py - INFO - Epoch 45/100:train_loss:27.753016, valid_loss:24.075053
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020

100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:48:03,003 - pytorch_modeler.py - INFO - Epoch 80/100:train_loss:27.054760, valid_loss:23.421691
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:49:08,217 - pytorch_modeler.py - INFO - Epoch 81/100:train_loss:27.181819, valid_loss:23.871891
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:50:13,442 - pytorch_modeler.py - INFO - Epoch 82/100:train_loss:27.053986, valid_loss:23.919342
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:51:18,649 - pytorch_modeler.py - INFO - Epoch 83/100:train_loss:26.934534, valid_loss:23.877729
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:52:23,856 - pytorch_modeler.py - INFO - Epoch 84/100:train_loss:27.024455, valid_loss:24.226474
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020-10-16 20:53:29,039 - pytorch_modeler.py - INFO - Epoch 85/100:train_loss:27.117873, valid_loss:23.367405
100%|██████████| 55/55 [00:18<00:00,  2.94it/s]
2020

elapsed time: 6567.677442789 [sec]
